In [1]:
%pip install pyannote.audio
%pip install numpy
%pip install google-generativeai python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
from dotenv import load_dotenv

load_dotenv()

HUGGING_FACE_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

In [ ]:
#토큰 사용시 권한 확인(✅ Read access to contents of all public gated repos you can access)
from pyannote.audio import Pipeline

pipeline = Pipeline.from_pretrained(
  "pyannote/speaker-diarization-3.1",
  use_auth_token=HUGGING_FACE_TOKEN
)

In [6]:
#연습용 HF 경로 확인
import os
from dotenv import load_dotenv
load_dotenv()

token = os.getenv("HUGGING_FACE_TOKEN")
print("Has token:", bool(token), "| length:", len(token) if token else None)
print("Head:", token[:7] if token else None)  # hf_로 시작해야 정상

Has token: True | length: 37
Head: hf_tBSv


In [9]:
import torch

# cuda가 사용 가능한 경우 cuda를 사용하도록 설정
if torch.cuda.is_available():
    pipeline.to(torch.device("cuda"))
    print('cuda is available')
else:
    print('cuda is not available')

cuda is not available


In [13]:
# run the pipeline on an audio file
# diarization = pipeline("audio.wav")
diarization = pipeline(r"C:\Users\전산2실 PC\Downloads\PRACTICE1\chap5\싼기타_비싼기타.mp3")

# dump the diarization output to disk using RTTM format
with open(r"C:\Users\전산2실 PC\Downloads\PRACTICE1\chap5\싼기타_비싼기타.rttm", "w", encoding='utf-8') as rttm:
    diarization.write_rttm(rttm)

c:\Users\전산2실 PC\Downloads\PRACTICE1\venv_new\Lib\site-packages\torchaudio\_backend\utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
c:\Users\전산2실 PC\Downloads\PRACTICE1\venv_new\Lib\site-packages\pyannote\audio\core\io.py:85: UserWarning: torchaudio._backend.utils.info has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more inform

RTTM 한줄 구조(화자 분할)
RTTM은 Rich Transcription Time Marked의 약자로, 오디오 파일의 내용을 시간 정보와 함께 기록하는 표준 텍스트 파일 형식입니다. 주로 화자 분할(Speaker Diarization), 음성 인식(Speech Recognition) 등 다양한 음성 처리 연구 및 평가에서 사용됩니다.

SPEAKER [파일명] 1 [시작 시간] [지속 시간] <NA> <NA> [스피커 레이블] <NA> <NA>
SPEAKER 싼기타_비싼기타 1 10.53 2.81 <NA> <NA> SPEAKER_00 <NA> <NA>

| 필드       | 값(예시)        | 의미                                                   |
| -------- | ------------ | ---------------------------------------------------- |
| 레코드 유형   | `SPEAKER`    | 레코드 타입. 화자 분할 결과는 항상 `SPEAKER`.                      |
| 파일명      | `싼기타_비싼기타`   | 처리된 오디오 파일의 식별자(보통 파일명).                             |
| 채널       | `1`          | 오디오 채널(보통 1 = 모노).                                   |
| 시작 시간(초) | `10.53`      | 해당 화자가 말을 **시작**한 시각(초).                             |
| 지속 시간(초) | `2.81`       | 해당 화자가 말을 한 **총 길이**(초).                             |
| 사용 안 함   | `<NA>`       | 사용하지 않는 필드(표준 자리 채움).                                |
| 사용 안 함   | `<NA>`       | 사용하지 않는 필드(표준 자리 채움).                                |
| 스피커 레이블  | `SPEAKER_00` | pyannote가 부여한 화자 ID (`SPEAKER_00`, `SPEAKER_01`, …). |
| 사용 안 함   | `<NA>`       | 사용하지 않는 필드(표준 자리 채움).                                |
| 사용 안 함   | `<NA>`       | 사용하지 않는 필드(표준 자리 채움).                                |



In [15]:
# RTTM을 CSV로 변환
import pandas as pd
rttm_path = r"C:\Users\전산2실 PC\Downloads\PRACTICE1\chap5\싼기타_비싼기타.rttm"

df_rttm = pd.read_csv(
    rttm_path,      # rttm 파일 경로
    sep=' ',        # 구분자는 띄어쓰기
    header=None,    # 헤더는 없음
    names=['type', 'file', 'chnl', 'start', 'duration', 'C1', 'C2', 'speaker_id', 'C3', 'C4'] 
)
df_rttm['end'] = df_rttm['start'] + df_rttm['duration']  # 종료 시간 계산
display(df_rttm)

type      file  chnl    start  duration  C1  C2  speaker_id  C3  C4  \
0   SPEAKER  싼기타_비싼기타     1    0.993     5.805 NaN NaN  SPEAKER_01 NaN NaN   
1   SPEAKER  싼기타_비싼기타     1    7.405     3.983 NaN NaN  SPEAKER_01 NaN NaN   
2   SPEAKER  싼기타_비싼기타     1   11.759     4.927 NaN NaN  SPEAKER_01 NaN NaN   
3   SPEAKER  싼기타_비싼기타     1   17.210    10.665 NaN NaN  SPEAKER_01 NaN NaN   
4   SPEAKER  싼기타_비싼기타     1   28.668     1.536 NaN NaN  SPEAKER_01 NaN NaN   
..      ...       ...   ...      ...       ...  ..  ..         ...  ..  ..   
83  SPEAKER  싼기타_비싼기타     1  414.481     2.970 NaN NaN  SPEAKER_00 NaN NaN   
84  SPEAKER  싼기타_비싼기타     1  417.755     3.476 NaN NaN  SPEAKER_01 NaN NaN   
85  SPEAKER  싼기타_비싼기타     1  423.644     0.776 NaN NaN  SPEAKER_00 NaN NaN   
86  SPEAKER  싼기타_비싼기타     1  424.741     3.527 NaN NaN  SPEAKER_00 NaN NaN   
87  SPEAKER  싼기타_비싼기타     1  428.504     0.844 NaN NaN  SPEAKER_00 NaN NaN   

        end  
0     6.798  
1    11.388  
2    16.686  
3    27.875  
4    30.204  
..      ...  
83  417.451  
84  421.231  
85  424.420  
86  428.268  
87  429.348  

[88 rows x 11 columns]

In [16]:
df_rttm["number"] = None	# number 열 만들고 None으로 초기화
df_rttm.at[0, "number"] = 0 # numberr 열의 0번째 인덱스 값을 0으로 설정

display(df_rttm)

type      file  chnl    start  duration  C1  C2  speaker_id  C3  C4  \
0   SPEAKER  싼기타_비싼기타     1    0.993     5.805 NaN NaN  SPEAKER_01 NaN NaN   
1   SPEAKER  싼기타_비싼기타     1    7.405     3.983 NaN NaN  SPEAKER_01 NaN NaN   
2   SPEAKER  싼기타_비싼기타     1   11.759     4.927 NaN NaN  SPEAKER_01 NaN NaN   
3   SPEAKER  싼기타_비싼기타     1   17.210    10.665 NaN NaN  SPEAKER_01 NaN NaN   
4   SPEAKER  싼기타_비싼기타     1   28.668     1.536 NaN NaN  SPEAKER_01 NaN NaN   
..      ...       ...   ...      ...       ...  ..  ..         ...  ..  ..   
83  SPEAKER  싼기타_비싼기타     1  414.481     2.970 NaN NaN  SPEAKER_00 NaN NaN   
84  SPEAKER  싼기타_비싼기타     1  417.755     3.476 NaN NaN  SPEAKER_01 NaN NaN   
85  SPEAKER  싼기타_비싼기타     1  423.644     0.776 NaN NaN  SPEAKER_00 NaN NaN   
86  SPEAKER  싼기타_비싼기타     1  424.741     3.527 NaN NaN  SPEAKER_00 NaN NaN   
87  SPEAKER  싼기타_비싼기타     1  428.504     0.844 NaN NaN  SPEAKER_00 NaN NaN   

        end number  
0     6.798      0  
1    11.388   None  
2    16.686   None  
3    27.875   None  
4    30.204   None  
..      ...    ...  
83  417.451   None  
84  421.231   None  
85  424.420   None  
86  428.268   None  
87  429.348   None  

[88 rows x 12 columns]

In [17]:
for i in range(1, len(df_rttm)):
    if df_rttm.at[i, "speaker_id"] != df_rttm.at[i-1, "speaker_id"]:
        df_rttm.at[i, "number"] = df_rttm.at[i-1, "number"] + 1
    else:
        df_rttm.at[i, "number"] = df_rttm.at[i-1, "number"]

display(df_rttm.head(10)) 

type      file  chnl   start  duration  C1  C2  speaker_id  C3  C4  \
0  SPEAKER  싼기타_비싼기타     1   0.993     5.805 NaN NaN  SPEAKER_01 NaN NaN   
1  SPEAKER  싼기타_비싼기타     1   7.405     3.983 NaN NaN  SPEAKER_01 NaN NaN   
2  SPEAKER  싼기타_비싼기타     1  11.759     4.927 NaN NaN  SPEAKER_01 NaN NaN   
3  SPEAKER  싼기타_비싼기타     1  17.210    10.665 NaN NaN  SPEAKER_01 NaN NaN   
4  SPEAKER  싼기타_비싼기타     1  28.668     1.536 NaN NaN  SPEAKER_01 NaN NaN   
5  SPEAKER  싼기타_비싼기타     1  32.414     0.759 NaN NaN  SPEAKER_00 NaN NaN   
6  SPEAKER  싼기타_비싼기타     1  33.545     3.561 NaN NaN  SPEAKER_00 NaN NaN   
7  SPEAKER  싼기타_비싼기타     1  37.628     3.763 NaN NaN  SPEAKER_00 NaN NaN   
8  SPEAKER  싼기타_비싼기타     1  41.611     1.097 NaN NaN  SPEAKER_00 NaN NaN   
9  SPEAKER  싼기타_비싼기타     1  41.645     0.810 NaN NaN  SPEAKER_01 NaN NaN   

      end number  
0   6.798      0  
1  11.388      0  
2  16.686      0  
3  27.875      0  
4  30.204      0  
5  33.173      1  
6  37.106      1  
7  41.391      1  
8  42.708      1  
9  42.455      2

In [22]:
import pandas as pd

df = pd.DataFrame({
    "class": ["A","A","A","B","B"],
    "height": [170, 165, 180, 160, 175],
    "weight": [60, 55, 70, 50, 65]
})
df

class  height  weight
0     A     170      60
1     A     165      55
2     A     180      70
3     B     160      50
4     B     175      65

In [23]:
result = df.groupby("class").agg(
    min_height = pd.NamedAgg(column="height", aggfunc="min"),   # height의 최소값
    avg_weight = pd.NamedAgg(column="weight", aggfunc="mean"),  # weight의 평균
)
print(result)

       min_height  avg_weight
class                        
A             165   61.666667
B             160   57.500000


In [25]:
result = df.groupby("class").agg(
    min_height = ("height", "min"),
    avg_weight = ("weight", "mean"),
)
print(result)

       min_height  avg_weight
class                        
A             165   61.666667
B             160   57.500000


groupby().agg() 에서 자주 쓰는 집계 함수(문자열) 정리

| 함수                  | 설명                         |
| ------------------- | -------------------------- |
| `count`             | 결측값(`NaN`)을 제외한 **데이터 개수** |
| `sum`               | **합계**                     |
| `mean`              | **평균**                     |
| `median`            | **중앙값**                    |
| `var`, `std`        | **분산**, **표준편차**           |
| `min`, `max`        | **최솟값**, **최댓값**           |
| `unique`, `nunique` | **고유값 목록**, **고유값 개수**     |
| `prod`              | **곱**(모든 값의 곱)             |
| `first`, `last`     | **첫 번째 값**, **마지막 값**      |


참고

대부분의 집계는 기본적으로 NaN을 무시합니다(mean, sum, …).

같은 열에 여러 함수를 동시에 적용하려면 리스트로: agg(['min','max','mean']).

다른 열마다 다른 함수를 적용하려면 딕셔너리/튜플 문법:
agg(height=('height','mean'), weight=('weight','max')).

In [29]:
#https://teddylee777.github.io/pandas/pandas-groupby/ 설명 참고 -> groupby와 agg 사용
df_rttm_grouped = df_rttm.groupby("number").agg(
    start=pd.NamedAgg(column='start', aggfunc='min'),
    end=pd.NamedAgg(column='end', aggfunc='max'),
    speaker_id=pd.NamedAgg(column='speaker_id', aggfunc='first')
)

display(df_rttm_grouped)

start      end  speaker_id
number                              
0         0.993   30.204  SPEAKER_01
1        32.414   42.708  SPEAKER_00
2        41.645   44.024  SPEAKER_01
3        45.813   67.109  SPEAKER_00
4        67.227   82.786  SPEAKER_01
5        84.659  102.564  SPEAKER_00
6       103.492  117.532  SPEAKER_01
7       119.759  138.676  SPEAKER_00
8       139.351  168.967  SPEAKER_01
9       170.907  192.321  SPEAKER_00
10      192.322  193.689  SPEAKER_01
11      192.760  193.503  SPEAKER_00
12      193.823  216.571  SPEAKER_01
13      218.579  238.120  SPEAKER_00
14      238.120  238.677  SPEAKER_01
15      238.188  239.352  SPEAKER_00
16      239.858  240.651  SPEAKER_01
17      240.297  241.006  SPEAKER_00
18      241.040  251.334  SPEAKER_01
19      253.696  271.550  SPEAKER_00
20      272.140  304.557  SPEAKER_01
21      306.970  326.022  SPEAKER_00
22      326.360  335.472  SPEAKER_01
23      337.548  355.925  SPEAKER_00
24      356.245  363.974  SPEAKER_01
25      365.982  381.423  SPEAKER_00
26      382.165  383.734  SPEAKER_01
27      385.726  397.184  SPEAKER_00
28      397.623  399.108  SPEAKER_01
29      401.437  406.601  SPEAKER_00
30      407.326  408.338  SPEAKER_01
31      410.093  417.451  SPEAKER_00
32      417.755  421.231  SPEAKER_01
33      423.644  429.348  SPEAKER_00

In [ ]:
#튜플형태로 구현

df_rttm_grouped2 = df_rttm.groupby("number").agg(
    start=('start','min'),
    end=('end', 'max'),
    speaker_id=('speaker_id','first'))

display(df_rttm_grouped2)

start      end  speaker_id
number                              
0         0.993   30.204  SPEAKER_01
1        32.414   42.708  SPEAKER_00
2        41.645   44.024  SPEAKER_01
3        45.813   67.109  SPEAKER_00
4        67.227   82.786  SPEAKER_01
5        84.659  102.564  SPEAKER_00
6       103.492  117.532  SPEAKER_01
7       119.759  138.676  SPEAKER_00
8       139.351  168.967  SPEAKER_01
9       170.907  192.321  SPEAKER_00
10      192.322  193.689  SPEAKER_01
11      192.760  193.503  SPEAKER_00
12      193.823  216.571  SPEAKER_01
13      218.579  238.120  SPEAKER_00
14      238.120  238.677  SPEAKER_01
15      238.188  239.352  SPEAKER_00
16      239.858  240.651  SPEAKER_01
17      240.297  241.006  SPEAKER_00
18      241.040  251.334  SPEAKER_01
19      253.696  271.550  SPEAKER_00
20      272.140  304.557  SPEAKER_01
21      306.970  326.022  SPEAKER_00
22      326.360  335.472  SPEAKER_01
23      337.548  355.925  SPEAKER_00
24      356.245  363.974  SPEAKER_01
25      365.982  381.423  SPEAKER_00
26      382.165  383.734  SPEAKER_01
27      385.726  397.184  SPEAKER_00
28      397.623  399.108  SPEAKER_01
29      401.437  406.601  SPEAKER_00
30      407.326  408.338  SPEAKER_01
31      410.093  417.451  SPEAKER_00
32      417.755  421.231  SPEAKER_01
33      423.644  429.348  SPEAKER_00

In [33]:
df_rttm_grouped["duration"] = df_rttm_grouped["end"] - df_rttm_grouped["start"]
df_rttm_grouped = df_rttm_grouped.reset_index(drop=True)
display(df_rttm_grouped)

start      end  speaker_id  duration
0     0.993   30.204  SPEAKER_01    29.211
1    32.414   42.708  SPEAKER_00    10.294
2    41.645   44.024  SPEAKER_01     2.379
3    45.813   67.109  SPEAKER_00    21.296
4    67.227   82.786  SPEAKER_01    15.559
5    84.659  102.564  SPEAKER_00    17.905
6   103.492  117.532  SPEAKER_01    14.040
7   119.759  138.676  SPEAKER_00    18.917
8   139.351  168.967  SPEAKER_01    29.616
9   170.907  192.321  SPEAKER_00    21.414
10  192.322  193.689  SPEAKER_01     1.367
11  192.760  193.503  SPEAKER_00     0.743
12  193.823  216.571  SPEAKER_01    22.748
13  218.579  238.120  SPEAKER_00    19.541
14  238.120  238.677  SPEAKER_01     0.557
15  238.188  239.352  SPEAKER_00     1.164
16  239.858  240.651  SPEAKER_01     0.793
17  240.297  241.006  SPEAKER_00     0.709
18  241.040  251.334  SPEAKER_01    10.294
19  253.696  271.550  SPEAKER_00    17.854
20  272.140  304.557  SPEAKER_01    32.417
21  306.970  326.022  SPEAKER_00    19.052
22  326.360  335.472  SPEAKER_01     9.112
23  337.548  355.925  SPEAKER_00    18.377
24  356.245  363.974  SPEAKER_01     7.729
25  365.982  381.423  SPEAKER_00    15.441
26  382.165  383.734  SPEAKER_01     1.569
27  385.726  397.184  SPEAKER_00    11.458
28  397.623  399.108  SPEAKER_01     1.485
29  401.437  406.601  SPEAKER_00     5.164
30  407.326  408.338  SPEAKER_01     1.012
31  410.093  417.451  SPEAKER_00     7.358
32  417.755  421.231  SPEAKER_01     3.476
33  423.644  429.348  SPEAKER_00     5.704